In [1]:
#---------------------------------------for jupyter lab---------------------------------------#
!pip install selenium
#!wget https://chromedriver.storage.googleapis.com/2.42/chromedriver_linux64.zip  && unzip chromedriver_linux64
    
#---------------------------------------for google colab--------------------------------------#
#!apt-get update
#!apt install chromium-chromedriver
#!cp /usr/lib/chromium-browser/chromedriver /usr/bin
#!pip install selenium

In [2]:
# import package
import pandas as pd
import numpy as np
import time as tm

from selenium import webdriver

import re
import os

from selenium.common.exceptions import StaleElementReferenceException
from collections import defaultdict
import gc
from datetime import timedelta

In [4]:
#--------------------------------------------for google drive mounting using google colab--------------------------------------------#

#from google.colab import drive # this will be our driver
#drive.mount('/gdrive/',force_remount=True)
#root = '/gdrive/My Drive/'
#colab_path = '/gdrive/../content/'
#drive_path = '/gdrive/../content/drive'
#TXT_path = '/gdrive/../content/drive/My Drive/TXTS2'
#drive.mount('/content/drive')
#os.chdir(TXT_path)
#!ls

In [6]:
def makeDriver (url):
    
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    
    try:
        
        ########for google colab##########  
        #driver = webdriver.Chrome('chromedriver', options=chrome_options)
        ########for jupyternotebook#######
        driver = webdriver.Chrome(options = chrome_options, executable_path = os.getcwd() + '/chromedriver')
        
        driver.get(url)
        
    except:
        
        print ("Error in making DRIVER")
        driver=None

    return driver

#############################################################################################################

def privacyPolicyURL (atag_text):
    
    for i in atag_text:
        
        href = i.get_attribute("href")

        if (href is not None and "privacy" in href):
            
            #print ("href in extracting the url of privacy policy = ",href)
            return (href)
        
    return None

In [ ]:
start_page=int (input("enter START page : "))
end_page=int (input("enter END page : "))

times=np.empty(end_page - start_page)
index=0

print (start_page," ====> ",end_page)

for page in range (start_page,end_page):
    
    f= open(os.path.join(os.getcwd(), "pages_extracted.txt"), "w+")
    f.write(str(page)+"\n")
    
    start_time = tm.time()
    driver = makeDriver ("https://www.quantcast.com/top-sites/GB/" + str(page))      #print (driver)

    if (driver is not None):

        elements = driver.find_elements_by_css_selector("a")

        print ("\npage # ", page, "     @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@\n")

        ii = 1
        for i in elements:
                href = i.get_attribute("href")
                indexstart = [(m.start(), m.end()) for m in re.finditer("measure/",href)]
                indexquestionmark = [(m.start(0), m.end(0)) for m in re.finditer("country=",href)]

                if len(indexstart) != 0 and len(indexquestionmark) != 0:

                    print ("----------------------------------------------------------Extracting URL-------------------------------------------------------------------------")
                    web_url_startindex = indexstart[0][1]
                    web_url_endindex = indexquestionmark[0][0] - 1
                    driver = makeDriver ("http://www." + href[web_url_startindex:])       #print ("driver = ",driver)

                    if driver is not None:

                        websitename = "www." + href[web_url_startindex:web_url_endindex]    #print ("website name = ",websitename)
                        atag_text = driver.find_elements_by_css_selector("a")
                        try:

                          href=privacyPolicyURL (atag_text)

                        except StaleElementReferenceException:

                          print ("stealing error")
                          continue

                        if href is not None:

                            driver = makeDriver (href)

                            if driver is not None:

                                print ("---------------------------------------------------------Extracting POLICY------------------------------------------------------------------------")
                                privacy_policy = ""
                                privacy_policy = driver.find_element_by_tag_name("body").text

                                if len(privacy_policy) > 0:

                                    print ("-----------------------------------------------------------Saving TXTs---------------------------------------------------------------------------")
                                    f= open(os.path.join(os.getcwd(), websitename + ".txt"), "w+")
                                    f.write(privacy_policy)
                                    print ("Saved site # ", ii, "  website name = ", websitename, "  Privacy_policy_URL = ", href)
                                    ii +=1

                        del atag_text

        del elements

    gc.collect()
    speed = tm.time() - start_time
    msg = "\nPage Execution took: %s secs\n" % timedelta(seconds=round(speed))
    print (msg)

    times[index]=speed
    index+=1

speed = 0
for time in times:
    speed+=time
msg = "\nAverage Execution took: %s secs\n" % timedelta (seconds = round (speed / (end_page - start_page)))
print(msg)

enter START page :  205
enter END page :  250


205  ====>  250

page #  205      @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@

----------------------------------------------------------Extracting URL-------------------------------------------------------------------------
----------------------------------------------------------Extracting URL-------------------------------------------------------------------------
----------------------------------------------------------Extracting URL-------------------------------------------------------------------------
----------------------------------------------------------Extracting URL-------------------------------------------------------------------------
----------------------------------------------------------Extracting URL-------------------------------------------------------------------------
----------------------------------------------------------Extracting URL---------------------------------------------------------------